In [4]:
import tensorflow as tf
import numpy as np
import glob



In [24]:
spiders = glob.glob('./Spider/*')
non_spiders = glob.glob('./NonSpider/*')

In [11]:
# print spiders

In [45]:
# step 1
# filenames = spiders #['im_01.jpg', 'im_02.jpg', 'im_03.jpg', 'im_04.jpg']

# step 2
fnlist = []
fnlist[0:len(spiders)-1] = spiders[:]

fnlist[len(spiders):len(spiders)+len(non_spiders)] = non_spiders[:]
# print spiders[:]
print np.shape(fnlist)
typelist = []
typelist[0:len(spiders)-1] = np.ones(len(spiders))
typelist[len(spiders):len(spiders)+len(non_spiders)] = - np.ones(len(non_spiders))
print np.shape(typelist)

spider_filename_queue = tf.train.string_input_producer(fnlist)

# step 3: read, decode and resize images
reader = tf.WholeFileReader()
filename, content = reader.read(spider_filename_queue)
image = tf.image.decode_jpeg(content, channels=3)
image = tf.cast(image, tf.float32)
imgSizeX = 256
imgSizeY = 256
spider_resized_image = tf.image.resize_images(image, imgSizeX, imgSizeY)

# # step 4: Batching
# spider_image_batch = tf.train.batch([[resized_image],np.ones(len(spiders))], batch_size=8)
                                     
# # step 2
# non_spider_filename_queue = tf.train.string_input_producer(non_spiders)

# # step 3: read, decode and resize images
# filename, content = reader.read(non_spider_filename_queue)
# image = tf.image.decode_jpeg(content, channels=3)
# image = tf.cast(image, tf.float32)
# imgSizeX = 256
# imgSizeY = 256
# non_spider_resized_image = tf.image.resize_images(image, imgSizeX, imgSizeY)

# # step 4: Batching
# # non_spider_image_batch = tf.train.batch([[resized_image],np.ones(len(non_spiders))], batch_size=8)

# total_images = spider_resized_image
# total_images.
# .append(non_spider_resized_image)
                                     
# total_batch = spider_image_batch
# total_batch.append(non_spider_image_batch)

(73,)
(73,)


In [51]:
# Parameters
learning_rate = 0.001
training_epochs = 15
# batch_size = 3
display_step = 1
n_input = typelist

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = imgSizeX*imgSizeY # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [53]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
#         total_batchS = len(total_batch)//batch_size
        # Loop over all batches
#         for i in range(total_batchS):
#             batch_x, batch_y = total_batch.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([optimizer, cost], feed_dict={x: spider_resized_image,y: typelist})
        # Compute average loss
        avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, or numpy ndarrays.